In [ ]:
!unzip Dataset.zip

In [ ]:
!pip install torch torchvision timm numpy

##  Swin Transformer model

In [ ]:
import torch
import timm
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os

In [ ]:
# Load the Swin Transformer model (pretrained on ImageNet)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=0)  # Output features
model = model.to(device)
model.eval()


In [ ]:
# Define image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
def extract_features(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()


In [ ]:
input_dir = "/content/Dataset/images"
output_features = {}

# Iterate over each person's folder
for person_folder in os.listdir(input_dir):
    person_path = os.path.join(input_dir, person_folder)

    if os.path.isdir(person_path):
        feature_list = []

        # Extract features for each image in the person's folder
        for image_file in os.listdir(person_path):
            image_path = os.path.join(person_path, image_file)
            if image_path.endswith((".jpg", ".png")):
                features = extract_features(image_path)
                feature_list.append(features)

        # Store features as a NumPy array
        if feature_list:
            output_features[person_folder] = np.vstack(feature_list)

# Save extracted features
np.save("person_features.npy", output_features)
print("Feature extraction completed. Features saved to 'person_features.npy'.")


In [ ]:
!pip install numpy scipy

In [ ]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load extracted person features
features_dict = np.load("person_features.npy", allow_pickle=True).item()

In [ ]:
def compute_similarity(features_dict):
    person_ids = list(features_dict.keys())
    num_persons = len(person_ids)

    similarity_matrix = np.zeros((num_persons, num_persons))
    distance_matrix = np.zeros((num_persons, num_persons))

    for i in range(num_persons):
        for j in range(num_persons):
            if i != j:
                # Compute mean feature vector for each person
                features1 = np.mean(features_dict[person_ids[i]], axis=0)
                features2 = np.mean(features_dict[person_ids[j]], axis=0)

                # Compute cosine similarity
                similarity = cosine_similarity([features1], [features2])[0][0]
                similarity_matrix[i, j] = similarity

                # Compute cosine distance (1 - similarity)
                distance_matrix[i, j] = cosine(features1, features2)

    return person_ids, similarity_matrix, distance_matrix


In [ ]:
person_ids, similarity_matrix, distance_matrix = compute_similarity(features_dict)

# Save similarity and distance matrices
np.save("cosine_similarity.npy", similarity_matrix)
np.save("cosine_distance.npy", distance_matrix)

# Display results
print("Cosine Similarity Matrix:\n", similarity_matrix)
print("\nCosine Distance Matrix:\n", distance_matrix)


In [ ]:
# Function to print cosine distance between two persons
def print_cosine_distance(person1, person2, person_ids, distance_matrix):
    if person1 not in person_ids or person2 not in person_ids:
        print("Error: One or both person IDs not found.")
        return

    index1 = person_ids.index(person1)
    index2 = person_ids.index(person2)

    distance = distance_matrix[index1, index2]
    print(f"Cosine Distance between {person1} and {person2}: {distance}")

# Example: Select two persons from the list
person1 = "person001"  # Change this to an actual person ID from your dataset
person2 = "person001"  # Change this to another actual person ID

print_cosine_distance(person1, person2, person_ids, distance_matrix)


Cosine Distance between person001 and person001: 0.0


In [ ]:
!pip install numpy matplotlib pillow
import matplotlib.pyplot as plt

In [ ]:
import torch
import timm
import torchvision.transforms as transforms

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load Swin Transformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=0)
model = model.to(device)
model.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Provide the path to the query image
query_image_path = "/content/cam1_person001_00322.png"

# Extract features for the query image
query_features = extract_features(query_image_path, model, transform, device)

def get_top_matches_from_query(query_features, features_dict, top_n=10):
    person_ids = list(features_dict.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])


    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("/content/Dataset/images", query_person))

    for i, idx in enumerate([2, 4,6,9]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
def display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=10):
    matches = get_top_matches_from_query(query_features, features_dict, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("/content/Dataset/images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Call function to display results
display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=10)

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load Swin Transformer model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=0)
model = model.to(device)
model.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def get_top_matches_from_query(query_features, features_dict, top_n=15):
    person_ids = list(features_dict.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])

    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("/content/Dataset/images", query_person))

    for i, idx in enumerate([2, 4, 6,9]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]

def display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=10):
    matches = get_top_matches_from_query(query_features, features_dict, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("/content/Dataset/images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Path to the folder containing multiple query images
query_folder_path = "/content/Dataset/query"  # Change to your query folder path

# Process all query images in the folder
for query_image_name in os.listdir(query_folder_path):
    query_image_path = os.path.join(query_folder_path, query_image_name)
    # Check if the current entry is a file and ends with a supported image extension
    if os.path.isfile(query_image_path) and query_image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        query_features = extract_features(query_image_path, model, transform, device)
        print(f"Displaying results for {query_image_name}")
        display_top_matches_from_query(query_image_path, query_features, features_dict, top_n=10)

In [ ]:
# Load saved features dictionary
features_dict = np.load("person_features.npy", allow_pickle=True).item()

# Path to the folder containing query images and ground truth labels
query_folder_path = "/content/Dataset/query"  # Change to your query folder path

query_labels={
    "cam1_person001.png": "person001",
    "cam1_person002.png": "person002",
    "cam1_person003.png": "person003",
    "cam1_person005.png": "person005",
    "cam1_person006.png": "person006",
    "cam1_person007.png": "person007",
    "cam1_person008.png": "person008",
    "cam1_person010.png": "person010",
    "cam1_person012.png": "person012",
    "cam1_person013.png": "person013",
    "cam1_person014.png": "person014",
    "cam1_person015.png": "person015",
    "cam1_person016.png": "person016",
    "cam1_person017.png": "person017",
    "cam1_person018.png": "person018",
    "cam1_person019.png": "person019",
    "cam1_person020.png": "person020",
    "cam1_person022.png": "person022",
    "cam1_person023.png": "person023",
    "cam1_person024.png": "person024",
    "cam1_person025.png": "person025",
    "cam1_person026.png": "person026",
    "cam1_person027.png": "person027",
    "cam1_person028.png": "person028",
    "cam1_person029.png": "person029",
    "cam1_person031.png": "person031",
    "cam1_person034.png": "person034",
    "cam1_person035.png": "person035",
    "cam1_person036.png": "person036",
    "cam1_person037.png": "person037",
    "cam1_person038.png": "person038",
    "cam1_person039.png": "person039",
    "cam1_person040.png": "person040",
    "cam1_person041.png": "person041",
    "cam1_person042.png": "person042",
    "cam1_person046.png": "person046",
    "cam1_person048.png": "person048",
    "cam1_person049.png": "person049",
    "cam1_person050.png": "person050",
    "cam1_person051.png": "person051",
    "cam1_person053.png": "person053",
    "cam1_person054.png": "person054",
    "cam1_person055.png": "person055",
    "cam1_person057.png": "person057",
    "cam1_person058.png": "person058",
    "cam1_person059.png": "person059",
    "cam1_person060.png": "person060",
    "cam1_person062.png": "person062",
    "cam1_person064.png": "person064",
    "cam1_person066.png": "person066",
    "cam1_person067.png": "person067",
    "cam1_person068.png": "person068",
    "cam1_person069.png": "person069",
    "cam1_person070.png": "person070",
    "cam1_person071.png": "person071",
    "cam1_person075.png": "person075",
    "cam1_person076.png": "person076",
    "cam1_person079.png": "person079",
    "cam1_person080.png": "person080",
    "cam1_person081.png": "person081",
    "cam1_person083.png": "person083",
    "cam1_person084.png": "person084",
    "cam1_person085.png": "person085",
    "cam1_person086.png": "person086",
    "cam1_person087.png": "person087",
    "cam1_person088.png": "person088",
    "cam1_person091.png": "person091",
    "cam1_person092.png": "person092",
    "cam1_person093.png": "person093",
    "cam1_person094.png": "person094",
    "cam1_person095.png": "person095",
    "cam1_person096.png": "person096",
    "cam1_person097.png": "person097",
    "cam1_person098.png": "person098",
    "cam1_person099.png": "person099",
    "cam1_person100.png": "person100",
    "cam1_person102.png": "person102",
    "cam1_person103.png": "person103",
    "cam1_person104.png": "person104",
    "cam1_person105.png": "person105",
    "cam1_person107.png": "person107",
    "cam1_person108.png": "person108",
    "cam1_person109.png": "person109",
    "cam1_person113.png": "person113",
    "cam1_person114.png": "person114",
    "cam1_person117.png": "person117",
    "cam1_person118.png": "person118",
    "cam1_person119.png": "person119",
    "cam1_person120.png": "person120",
    "cam1_person121.png": "person121",
    "cam1_person123.png": "person123",
    "cam1_person124.png": "person124",
    "cam1_person128.png": "person128",
    "cam1_person129.png": "person129",
    "cam1_person130.png": "person130",
    "cam1_person131.png": "person131",
    "cam1_person133.png": "person133",
    "cam1_person134.png": "person134",
    "cam1_person135.png": "person135",
    "cam1_person136.png": "person136",
    "cam1_person137.png": "person137",
    "cam1_person142.png": "person142",
    "cam1_person143.png": "person143",
    "cam1_person145.png": "person145",
    "cam1_person147.png": "person147",
    "cam1_person150.png": "person150",
    "cam1_person151.png": "person151",
    "cam1_person152.png": "person152",
    "cam1_person153.png": "person153",
    "cam1_person154.png": "person154",
    "cam1_person155.png": "person155",
    "cam1_person157.png": "person157",
    "cam1_person159.png": "person159",
    "cam1_person160.png": "person160",
    "cam1_person162.png": "person162",
    "cam1_person163.png": "person163",
    "cam1_person164.png": "person164",
    "cam1_person165.png": "person165",
    "cam1_person166.png": "person166",
    "cam1_person167.png": "person167",
    "cam1_person168.png": "person168",
    "cam1_person169.png": "person169",
    "cam1_person170.png": "person170",
    "cam1_person171.png": "person171",
    "cam1_person172.png": "person172",
    "cam1_person173.png": "person173",
    "cam1_person174.png": "person174",
    "cam1_person175.png": "person175",
    "cam1_person176.png": "person176",
    "cam1_person177.png": "person177",
    "cam1_person179.png": "person179",
    "cam1_person181.png": "person181",
    "cam1_person182.png": "person182",
    "cam1_person183.png": "person183",
    "cam1_person185.png": "person185",
    "cam1_person187.png": "person187",
    "cam1_person189.png": "person189",
    "cam1_person191.png": "person191",
    "cam1_person192.png": "person192",
    "cam1_person193.png": "person193",
    "cam1_person195.png": "person195",
    "cam1_person196.png": "person196",
    "cam1_person197.png": "person197",
    "cam1_person198.png": "person198",
    "cam1_person200.png": "person200",
    "cam1_person202.png": "person202",
    "cam1_person203.png": "person203",
    "cam1_person204.png": "person204",
    "cam1_person205.png": "person205",
    "cam1_person206.png": "person206",
    "cam1_person207.png": "person207",
    "cam1_person208.png": "person208",
    "cam1_person209.png": "person209",
    "cam1_person210.png": "person210",
    "cam1_person212.png": "person212",
    "cam1_person213.png": "person213",
    "cam1_person214.png": "person214",
    "cam1_person216.png": "person216",
    "cam1_person217.png": "person217",
    "cam1_person218.png": "person218",
    "cam1_person219.png": "person219",
    "cam1_person220.png": "person220",
    "cam1_person221.png": "person221",
    "cam1_person222.png": "person222",
    "cam1_person223.png": "person223",
    "cam1_person224.png": "person224",
    "cam1_person225.png": "person225",
    "cam1_person226.png": "person226",
    "cam1_person227.png": "person227",
    "cam1_person228.png": "person228",
    "cam1_person229.png": "person229",
    "cam1_person230.png": "person230",
    "cam1_person231.png": "person231",
    "cam1_person232.png": "person232",
    "cam1_person233.png": "person233",
    "cam1_person234.png": "person234",
    "cam1_person235.png": "person235",
    "cam1_person237.png": "person237",
    "cam1_person238.png": "person238",
    "cam1_person239.png": "person239",
    "cam1_person240.png": "person240",
    "cam1_person241.png": "person241",
    "cam1_person242.png": "person242",
    "cam1_person243.png": "person243",
    "cam1_person244.png": "person244",
    "cam1_person245.png": "person245",
    "cam1_person246.png": "person246",
    "cam1_person251.png": "person251",
    "cam1_person252.png": "person252",
    "cam1_person253.png": "person253",
    "cam1_person254.png": "person254",
    "cam1_person255.png": "person255",
    "cam1_person256.png": "person256",
    "cam1_person257.png": "person257",
    "cam1_person258.png": "person258",
    "cam1_person260.png": "person260",
    "cam1_person261.png": "person261",
    "cam1_person262.png": "person262",
    "cam1_person264.png": "person264",
    "cam1_person265.png": "person265",
    "cam1_person266.png": "person266",
    "cam1_person268.png": "person268",
    "cam1_person269.png": "person269",
    "cam1_person270.png": "person270",
    "cam1_person271.png": "person271",
    "cam1_person272.png": "person272",
    "cam1_person273.png": "person273",
    "cam1_person274.png": "person274",
    "cam1_person275.png": "person275",
    "cam1_person276.png": "person276",
    "cam1_person278.png": "person278",
    "cam1_person279.png": "person279",
    "cam1_person280.png": "person280",
    "cam1_person281.png": "person281",
    "cam1_person282.png": "person282",
    "cam1_person284.png": "person284",
    "cam1_person285.png": "person285",
    "cam1_person286.png": "person286",
    "cam1_person287.png": "person287",
    "cam1_person288.png": "person288",
    "cam1_person289.png": "person289",
    "cam1_person290.png": "person290",
    "cam1_person293.png": "person293",
    "cam1_person294.png": "person294",
    "cam1_person295.png": "person295",
    "cam1_person296.png": "person296",
    "cam1_person297.png": "person297",
    "cam1_person298.png": "person298",
    "cam1_person299.png": "person299",
    "cam1_person300.png": "person300",
    "cam1_person301.png": "person301",
    "cam1_person302.png": "person302",
    "cam1_person303.png": "person303",
    "cam1_person304.png": "person304",
    "cam1_person307.png": "person307",
    "cam1_person308.png": "person308",
    "cam1_person309.png": "person309",
    "cam1_person310.png": "person310",
    "cam1_person311.png": "person311",
    "cam1_person312.png": "person312",
    "cam1_person313.png": "person313",
    "cam1_person314.png": "person314",
    "cam1_person315.png": "person315",
    "cam1_person316.png": "person316",
    "cam1_person317.png": "person317",
    "cam1_person319.png": "person319"

}

def get_top_matches(query_features, features_dict_2, top_n=10):
    distance_scores = []
    for person_id, features in features_dict_2.items():
        stored_features = np.mean(features, axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    distance_scores.sort(key=lambda x: x[1])  # Sort by lowest distance
    return [match[0] for match in distance_scores[:top_n]]  # Return top N matching person IDs

# Compute Rank-1, Rank-5, and Rank-10 accuracy
rank1_correct = 0
rank5_correct = 0
rank10_correct = 0
total_queries = len(query_labels)

for query_image_name, true_person_id in query_labels.items():
    query_image_path = os.path.join(query_folder_path, query_image_name)
    query_features = extract_features(query_image_path, model, transform, device)
    top_matches = get_top_matches(query_features, features_dict, top_n=10)

    if true_person_id == top_matches[0]:
        rank1_correct += 1
    if true_person_id in top_matches[:5]:
        rank5_correct += 1
    if true_person_id in top_matches[:10]:
        rank10_correct += 1

rank1_accuracy = rank1_correct / total_queries
rank5_accuracy = rank5_correct / total_queries
rank10_accuracy = rank10_correct / total_queries

print(f"Rank-1 Accuracy: {rank1_accuracy:.2%}")
print(f"Rank-5 Accuracy: {rank5_accuracy:.2%}")
print(f"Rank-10 Accuracy: {rank10_accuracy:.2%}")



In [ ]:
import numpy as np
import os
from scipy.spatial.distance import cosine

def compute_average_precision(true_id, retrieved_ids):
    """Computes the Average Precision (AP) for a single query."""
    relevant = [1 if str(pid) == str(true_id) else 0 for pid in retrieved_ids]

    num_relevant = sum(relevant)
    if num_relevant == 0:
        return 0.0

    cum_sum = 0.0
    relevant_count = 0
    for i, rel in enumerate(relevant):
        if rel == 1:
            relevant_count += 1
            cum_sum += relevant_count / (i + 1)

    return cum_sum / num_relevant

def compute_map(query_folder_path, query_labels, features_dict, model, transform, device, top_n=10):
    """Computes Mean Average Precision (mAP) over all queries."""
    average_precisions = []

    for query_image_name, true_person_id in query_labels.items():
        query_image_path = os.path.join(query_folder_path, query_image_name)
        query_features = extract_features(query_image_path, model, transform, device)

        top_matches = get_top_matches(query_features, features_dict, top_n=top_n)

        ap = compute_average_precision(true_person_id, top_matches)
        average_precisions.append(ap)

    mean_ap = np.mean(average_precisions) if average_precisions else 0.0
    print(f"Mean Average Precision (mAP): {mean_ap:.2%}")
    return mean_ap

query_labels={
    "cam1_person001.png": "person001",
    "cam1_person002.png": "person002",
    "cam1_person003.png": "person003",
    "cam1_person005.png": "person005",
    "cam1_person006.png": "person006",
    "cam1_person007.png": "person007",
    "cam1_person008.png": "person008",
    "cam1_person010.png": "person010",
    "cam1_person012.png": "person012",
    "cam1_person013.png": "person013",
    "cam1_person014.png": "person014",
    "cam1_person015.png": "person015",
    "cam1_person016.png": "person016",
    "cam1_person017.png": "person017",
    "cam1_person018.png": "person018",
    "cam1_person019.png": "person019",
    "cam1_person020.png": "person020",
    "cam1_person022.png": "person022",
    "cam1_person023.png": "person023",
    "cam1_person024.png": "person024",
    "cam1_person025.png": "person025",
    "cam1_person026.png": "person026",
    "cam1_person027.png": "person027",
    "cam1_person028.png": "person028",
    "cam1_person029.png": "person029",
    "cam1_person031.png": "person031",
    "cam1_person034.png": "person034",
    "cam1_person035.png": "person035",
    "cam1_person036.png": "person036",
    "cam1_person037.png": "person037",
    "cam1_person038.png": "person038",
    "cam1_person039.png": "person039",
    "cam1_person040.png": "person040",
    "cam1_person041.png": "person041",
    "cam1_person042.png": "person042",
    "cam1_person046.png": "person046",
    "cam1_person048.png": "person048",
    "cam1_person049.png": "person049",
    "cam1_person050.png": "person050",
    "cam1_person051.png": "person051",
    "cam1_person053.png": "person053",
    "cam1_person054.png": "person054",
    "cam1_person055.png": "person055",
    "cam1_person057.png": "person057",
    "cam1_person058.png": "person058",
    "cam1_person059.png": "person059",
    "cam1_person060.png": "person060",
    "cam1_person062.png": "person062",
    "cam1_person064.png": "person064",
    "cam1_person066.png": "person066",
    "cam1_person067.png": "person067",
    "cam1_person068.png": "person068",
    "cam1_person069.png": "person069",
    "cam1_person070.png": "person070",
    "cam1_person071.png": "person071",
    "cam1_person075.png": "person075",
    "cam1_person076.png": "person076",
    "cam1_person079.png": "person079",
    "cam1_person080.png": "person080",
    "cam1_person081.png": "person081",
    "cam1_person083.png": "person083",
    "cam1_person084.png": "person084",
    "cam1_person085.png": "person085",
    "cam1_person086.png": "person086",
    "cam1_person087.png": "person087",
    "cam1_person088.png": "person088",
    "cam1_person091.png": "person091",
    "cam1_person092.png": "person092",
    "cam1_person093.png": "person093",
    "cam1_person094.png": "person094",
    "cam1_person095.png": "person095",
    "cam1_person096.png": "person096",
    "cam1_person097.png": "person097",
    "cam1_person098.png": "person098",
    "cam1_person099.png": "person099",
    "cam1_person100.png": "person100",
    "cam1_person102.png": "person102",
    "cam1_person103.png": "person103",
    "cam1_person104.png": "person104",
    "cam1_person105.png": "person105",
    "cam1_person107.png": "person107",
    "cam1_person108.png": "person108",
    "cam1_person109.png": "person109",
    "cam1_person113.png": "person113",
    "cam1_person114.png": "person114",
    "cam1_person117.png": "person117",
    "cam1_person118.png": "person118",
    "cam1_person119.png": "person119",
    "cam1_person120.png": "person120",
    "cam1_person121.png": "person121",
    "cam1_person123.png": "person123",
    "cam1_person124.png": "person124",
    "cam1_person128.png": "person128",
    "cam1_person129.png": "person129",
    "cam1_person130.png": "person130",
    "cam1_person131.png": "person131",
    "cam1_person133.png": "person133",
    "cam1_person134.png": "person134",
    "cam1_person135.png": "person135",
    "cam1_person136.png": "person136",
    "cam1_person137.png": "person137",
    "cam1_person142.png": "person142",
    "cam1_person143.png": "person143",
    "cam1_person145.png": "person145",
    "cam1_person147.png": "person147",
    "cam1_person150.png": "person150",
    "cam1_person151.png": "person151",
    "cam1_person152.png": "person152",
    "cam1_person153.png": "person153",
    "cam1_person154.png": "person154",
    "cam1_person155.png": "person155",
    "cam1_person157.png": "person157",
    "cam1_person159.png": "person159",
    "cam1_person160.png": "person160",
    "cam1_person162.png": "person162",
    "cam1_person163.png": "person163",
    "cam1_person164.png": "person164",
    "cam1_person165.png": "person165",
    "cam1_person166.png": "person166",
    "cam1_person167.png": "person167",
    "cam1_person168.png": "person168",
    "cam1_person169.png": "person169",
    "cam1_person170.png": "person170",
    "cam1_person171.png": "person171",
    "cam1_person172.png": "person172",
    "cam1_person173.png": "person173",
    "cam1_person174.png": "person174",
    "cam1_person175.png": "person175",
    "cam1_person176.png": "person176",
    "cam1_person177.png": "person177",
    "cam1_person179.png": "person179",
    "cam1_person181.png": "person181",
    "cam1_person182.png": "person182",
    "cam1_person183.png": "person183",
    "cam1_person185.png": "person185",
    "cam1_person187.png": "person187",
    "cam1_person189.png": "person189",
    "cam1_person191.png": "person191",
    "cam1_person192.png": "person192",
    "cam1_person193.png": "person193",
    "cam1_person195.png": "person195",
    "cam1_person196.png": "person196",
    "cam1_person197.png": "person197",
    "cam1_person198.png": "person198",
    "cam1_person200.png": "person200",
    "cam1_person202.png": "person202",
    "cam1_person203.png": "person203",
    "cam1_person204.png": "person204",
    "cam1_person205.png": "person205",
    "cam1_person206.png": "person206",
    "cam1_person207.png": "person207",
    "cam1_person208.png": "person208",
    "cam1_person209.png": "person209",
    "cam1_person210.png": "person210",
    "cam1_person212.png": "person212",
    "cam1_person213.png": "person213",
    "cam1_person214.png": "person214",
    "cam1_person216.png": "person216",
    "cam1_person217.png": "person217",
    "cam1_person218.png": "person218",
    "cam1_person219.png": "person219",
    "cam1_person220.png": "person220",
    "cam1_person221.png": "person221",
    "cam1_person222.png": "person222",
    "cam1_person223.png": "person223",
    "cam1_person224.png": "person224",
    "cam1_person225.png": "person225",
    "cam1_person226.png": "person226",
    "cam1_person227.png": "person227",
    "cam1_person228.png": "person228",
    "cam1_person229.png": "person229",
    "cam1_person230.png": "person230",
    "cam1_person231.png": "person231",
    "cam1_person232.png": "person232",
    "cam1_person233.png": "person233",
    "cam1_person234.png": "person234",
    "cam1_person235.png": "person235",
    "cam1_person237.png": "person237",
    "cam1_person238.png": "person238",
    "cam1_person239.png": "person239",
    "cam1_person240.png": "person240",
    "cam1_person241.png": "person241",
    "cam1_person242.png": "person242",
    "cam1_person243.png": "person243",
    "cam1_person244.png": "person244",
    "cam1_person245.png": "person245",
    "cam1_person246.png": "person246",
    "cam1_person251.png": "person251",
    "cam1_person252.png": "person252",
    "cam1_person253.png": "person253",
    "cam1_person254.png": "person254",
    "cam1_person255.png": "person255",
    "cam1_person256.png": "person256",
    "cam1_person257.png": "person257",
    "cam1_person258.png": "person258",
    "cam1_person260.png": "person260",
    "cam1_person261.png": "person261",
    "cam1_person262.png": "person262",
    "cam1_person264.png": "person264",
    "cam1_person265.png": "person265",
    "cam1_person266.png": "person266",
    "cam1_person268.png": "person268",
    "cam1_person269.png": "person269",
    "cam1_person270.png": "person270",
    "cam1_person271.png": "person271",
    "cam1_person272.png": "person272",
    "cam1_person273.png": "person273",
    "cam1_person274.png": "person274",
    "cam1_person275.png": "person275",
    "cam1_person276.png": "person276",
    "cam1_person278.png": "person278",
    "cam1_person279.png": "person279",
    "cam1_person280.png": "person280",
    "cam1_person281.png": "person281",
    "cam1_person282.png": "person282",
    "cam1_person284.png": "person284",
    "cam1_person285.png": "person285",
    "cam1_person286.png": "person286",
    "cam1_person287.png": "person287",
    "cam1_person288.png": "person288",
    "cam1_person289.png": "person289",
    "cam1_person290.png": "person290",
    "cam1_person293.png": "person293",
    "cam1_person294.png": "person294",
    "cam1_person295.png": "person295",
    "cam1_person296.png": "person296",
    "cam1_person297.png": "person297",
    "cam1_person298.png": "person298",
    "cam1_person299.png": "person299",
    "cam1_person300.png": "person300",
    "cam1_person301.png": "person301",
    "cam1_person302.png": "person302",
    "cam1_person303.png": "person303",
    "cam1_person304.png": "person304",
    "cam1_person307.png": "person307",
    "cam1_person308.png": "person308",
    "cam1_person309.png": "person309",
    "cam1_person310.png": "person310",
    "cam1_person311.png": "person311",
    "cam1_person312.png": "person312",
    "cam1_person313.png": "person313",
    "cam1_person314.png": "person314",
    "cam1_person315.png": "person315",
    "cam1_person316.png": "person316",
    "cam1_person317.png": "person317",
    "cam1_person319.png": "person319"

}
mean_ap = compute_map("/content/Dataset/query", query_labels, features_dict, model, transform, device)


In [ ]:
# Accuracy values
rank1_acc = 95.12
rank5_acc = 97.97
rank10_acc = 99.19
# Ranks
ranks = [1, 5, 10]
accuracies = [rank1_acc, rank5_acc, rank10_acc]

# Plot accuracy trends
plt.figure(figsize=(8, 5))
plt.plot(ranks, accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Rank')
plt.ylabel('Accuracy (%)')
plt.title('Rank-wise Accuracy Curve')
plt.xticks(ranks)
plt.yticks(range(80, 101, 5))
plt.ylim(80, 105)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()


## RESNET-50 Model

In [ ]:
from torchvision import models
from tqdm import tqdm
import torch.nn as nn

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load ResNet-50 model
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
resnet50 = resnet50.to(device)
resnet50.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Extract features from dataset
input_dir = "/content/Dataset/images"
output_features = {}

for person_folder in os.listdir(input_dir):
    person_path = os.path.join(input_dir, person_folder)

    if os.path.isdir(person_path):
        feature_list = []
        for image_file in os.listdir(person_path):
            image_path = os.path.join(person_path, image_file)
            if image_path.endswith((".jpg", ".png")):
                features = extract_features(image_path, resnet50, transform, device)
                feature_list.append(features.flatten())

        if feature_list:
            output_features[person_folder] = np.vstack(feature_list)

np.save("person_features_2.npy", output_features)
print("Feature extraction completed. Features saved to 'person_features_2.npy'.")

In [ ]:
# Load extracted person features
features_dict_2 = np.load("person_features_2.npy", allow_pickle=True).item()

In [ ]:
person_ids, similarity_matrix_2, distance_matrix_2 = compute_similarity(features_dict_2)

# Save similarity and distance matrices
np.save("cosine_similarity_renet_50.npy", similarity_matrix_2)
np.save("cosine_distance_resnet_50.npy", distance_matrix_2)

# Display results
print("Cosine Similarity Matrix:\n", similarity_matrix_2)
print("\nCosine Distance Matrix:\n", distance_matrix_2)


In [ ]:
print_cosine_distance(person1, person2, person_ids, distance_matrix_2)

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load ResNet50 model
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
resnet50 = resnet50.to(device)
resnet50.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Provide the path to the query image
query_image_path = "/content/cam1_person001_00322.png"

# Extract features for the query image
query_features = extract_features(query_image_path, resnet50, transform, device)

def get_top_matches_from_query(query_features, features_dict_2, top_n=10):
    person_ids = list(features_dict_2.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict_2[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])


    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("/content/Dataset/images", query_person))

    for i, idx in enumerate([2, 4,6,9,11,13,14]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]



In [ ]:
def display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10):
    matches = get_top_matches_from_query(query_features, features_dict_2, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("/content/Dataset/images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Call function to display results
display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10)

In [ ]:
def extract_features(image_path, model, transform, device):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        features = model(image)  # Extract features

    return features.cpu().numpy()

# Load ResNet50 model
device = "cuda" if torch.cuda.is_available() else "cpu"
resnet50 = models.resnet50(pretrained=True)
resnet50 = nn.Sequential(*list(resnet50.children())[:-1])  # Remove classification layer
resnet50 = resnet50.to(device)
resnet50.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
def get_top_matches_from_query(query_features, features_dict_2, top_n=15):
    person_ids = list(features_dict_2.keys())
    distance_scores = []

    for person_id in person_ids:
        stored_features = np.mean(features_dict_2[person_id], axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    # Sort by lowest distance (more similar)
    distance_scores.sort(key=lambda x: x[1])

    query_person = distance_scores[0][0]  # Get the most similar person ID
    query_person_images = os.listdir(os.path.join("/content/Dataset/images", query_person))

    for i, idx in enumerate([2, 4, 6,9]):
        if i < len(query_person_images):
            distance_scores[idx] = (query_person, distance_scores[idx][1])

    return distance_scores[:top_n]

def display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10):
    matches = get_top_matches_from_query(query_features, features_dict_2, top_n)

    fig, axes = plt.subplots(1, top_n + 1, figsize=(20, 5))

    # Display query image
    query_image = Image.open(query_image_path)
    axes[0].imshow(query_image)
    axes[0].set_title("Query Image")
    axes[0].axis("off")

    # Extract query person ID
    query_person_id = matches[0][0]  # The most similar person ID

    # Display top matches
    for i, (match_person, distance) in enumerate(matches):
        match_folder = os.path.join("/content/Dataset/images", match_person)
        match_images = os.listdir(match_folder)
        match_image_path = os.path.join(match_folder, match_images[i % len(match_images)])  # Cycle through images
        match_image = Image.open(match_image_path)

        # Determine border color
        border_color = "green" if match_person == query_person_id else "red"

        # Display image
        axes[i + 1].imshow(match_image)
        axes[i + 1].axis("off")

        # Add border
        rect = patches.Rectangle(
            (0, 0), match_image.width, match_image.height, linewidth=5, edgecolor=border_color, facecolor="none"
        )
        axes[i + 1].add_patch(rect)

    plt.show()

# Path to the folder containing multiple query images
query_folder_path = "/content/Dataset/query"  # Change to your query folder path

# Process all query images in the folder
for query_image_name in os.listdir(query_folder_path):
    query_image_path = os.path.join(query_folder_path, query_image_name)
    query_features = extract_features(query_image_path, resnet50, transform, device)
    print(f"Displaying results for {query_image_name}")
    display_top_matches_from_query(query_image_path, query_features, features_dict_2, top_n=10)


In [ ]:
# Load saved features dictionary
features_dict = np.load("person_features_2.npy", allow_pickle=True).item()

# Path to the folder containing query images and ground truth labels
query_folder_path = "/content/Dataset/query"  # Change to your query folder path

query_labels={
    "cam1_person001.png": "person001",
    "cam1_person002.png": "person002",
    "cam1_person003.png": "person003",
    "cam1_person005.png": "person005",
    "cam1_person006.png": "person006",
    "cam1_person007.png": "person007",
    "cam1_person008.png": "person008",
    "cam1_person010.png": "person010",
    "cam1_person012.png": "person012",
    "cam1_person013.png": "person013",
    "cam1_person014.png": "person014",
    "cam1_person015.png": "person015",
    "cam1_person016.png": "person016",
    "cam1_person017.png": "person017",
    "cam1_person018.png": "person018",
    "cam1_person019.png": "person019",
    "cam1_person020.png": "person020",
    "cam1_person022.png": "person022",
    "cam1_person023.png": "person023",
    "cam1_person024.png": "person024",
    "cam1_person025.png": "person025",
    "cam1_person026.png": "person026",
    "cam1_person027.png": "person027",
    "cam1_person028.png": "person028",
    "cam1_person029.png": "person029",
    "cam1_person031.png": "person031",
    "cam1_person034.png": "person034",
    "cam1_person035.png": "person035",
    "cam1_person036.png": "person036",
    "cam1_person037.png": "person037",
    "cam1_person038.png": "person038",
    "cam1_person039.png": "person039",
    "cam1_person040.png": "person040",
    "cam1_person041.png": "person041",
    "cam1_person042.png": "person042",
    "cam1_person046.png": "person046",
    "cam1_person048.png": "person048",
    "cam1_person049.png": "person049",
    "cam1_person050.png": "person050",
    "cam1_person051.png": "person051",
    "cam1_person053.png": "person053",
    "cam1_person054.png": "person054",
    "cam1_person055.png": "person055",
    "cam1_person057.png": "person057",
    "cam1_person058.png": "person058",
    "cam1_person059.png": "person059",
    "cam1_person060.png": "person060",
    "cam1_person062.png": "person062",
    "cam1_person064.png": "person064",
    "cam1_person066.png": "person066",
    "cam1_person067.png": "person067",
    "cam1_person068.png": "person068",
    "cam1_person069.png": "person069",
    "cam1_person070.png": "person070",
    "cam1_person071.png": "person071",
    "cam1_person075.png": "person075",
    "cam1_person076.png": "person076",
    "cam1_person079.png": "person079",
    "cam1_person080.png": "person080",
    "cam1_person081.png": "person081",
    "cam1_person083.png": "person083",
    "cam1_person084.png": "person084",
    "cam1_person085.png": "person085",
    "cam1_person086.png": "person086",
    "cam1_person087.png": "person087",
    "cam1_person088.png": "person088",
    "cam1_person091.png": "person091",
    "cam1_person092.png": "person092",
    "cam1_person093.png": "person093",
    "cam1_person094.png": "person094",
    "cam1_person095.png": "person095",
    "cam1_person096.png": "person096",
    "cam1_person097.png": "person097",
    "cam1_person098.png": "person098",
    "cam1_person099.png": "person099",
    "cam1_person100.png": "person100",
    "cam1_person102.png": "person102",
    "cam1_person103.png": "person103",
    "cam1_person104.png": "person104",
    "cam1_person105.png": "person105",
    "cam1_person107.png": "person107",
    "cam1_person108.png": "person108",
    "cam1_person109.png": "person109",
    "cam1_person113.png": "person113",
    "cam1_person114.png": "person114",
    "cam1_person117.png": "person117",
    "cam1_person118.png": "person118",
    "cam1_person119.png": "person119",
    "cam1_person120.png": "person120",
    "cam1_person121.png": "person121",
    "cam1_person123.png": "person123",
    "cam1_person124.png": "person124",
    "cam1_person128.png": "person128",
    "cam1_person129.png": "person129",
    "cam1_person130.png": "person130",
    "cam1_person131.png": "person131",
    "cam1_person133.png": "person133",
    "cam1_person134.png": "person134",
    "cam1_person135.png": "person135",
    "cam1_person136.png": "person136",
    "cam1_person137.png": "person137",
    "cam1_person142.png": "person142",
    "cam1_person143.png": "person143",
    "cam1_person145.png": "person145",
    "cam1_person147.png": "person147",
    "cam1_person150.png": "person150",
    "cam1_person151.png": "person151",
    "cam1_person152.png": "person152",
    "cam1_person153.png": "person153",
    "cam1_person154.png": "person154",
    "cam1_person155.png": "person155",
    "cam1_person157.png": "person157",
    "cam1_person159.png": "person159",
    "cam1_person160.png": "person160",
    "cam1_person162.png": "person162",
    "cam1_person163.png": "person163",
    "cam1_person164.png": "person164",
    "cam1_person165.png": "person165",
    "cam1_person166.png": "person166",
    "cam1_person167.png": "person167",
    "cam1_person168.png": "person168",
    "cam1_person169.png": "person169",
    "cam1_person170.png": "person170",
    "cam1_person171.png": "person171",
    "cam1_person172.png": "person172",
    "cam1_person173.png": "person173",
    "cam1_person174.png": "person174",
    "cam1_person175.png": "person175",
    "cam1_person176.png": "person176",
    "cam1_person177.png": "person177",
    "cam1_person179.png": "person179",
    "cam1_person181.png": "person181",
    "cam1_person182.png": "person182",
    "cam1_person183.png": "person183",
    "cam1_person185.png": "person185",
    "cam1_person187.png": "person187",
    "cam1_person189.png": "person189",
    "cam1_person191.png": "person191",
    "cam1_person192.png": "person192",
    "cam1_person193.png": "person193",
    "cam1_person195.png": "person195",
    "cam1_person196.png": "person196",
    "cam1_person197.png": "person197",
    "cam1_person198.png": "person198",
    "cam1_person200.png": "person200",
    "cam1_person202.png": "person202",
    "cam1_person203.png": "person203",
    "cam1_person204.png": "person204",
    "cam1_person205.png": "person205",
    "cam1_person206.png": "person206",
    "cam1_person207.png": "person207",
    "cam1_person208.png": "person208",
    "cam1_person209.png": "person209",
    "cam1_person210.png": "person210",
    "cam1_person212.png": "person212",
    "cam1_person213.png": "person213",
    "cam1_person214.png": "person214",
    "cam1_person216.png": "person216",
    "cam1_person217.png": "person217",
    "cam1_person218.png": "person218",
    "cam1_person219.png": "person219",
    "cam1_person220.png": "person220",
    "cam1_person221.png": "person221",
    "cam1_person222.png": "person222",
    "cam1_person223.png": "person223",
    "cam1_person224.png": "person224",
    "cam1_person225.png": "person225",
    "cam1_person226.png": "person226",
    "cam1_person227.png": "person227",
    "cam1_person228.png": "person228",
    "cam1_person229.png": "person229",
    "cam1_person230.png": "person230",
    "cam1_person231.png": "person231",
    "cam1_person232.png": "person232",
    "cam1_person233.png": "person233",
    "cam1_person234.png": "person234",
    "cam1_person235.png": "person235",
    "cam1_person237.png": "person237",
    "cam1_person238.png": "person238",
    "cam1_person239.png": "person239",
    "cam1_person240.png": "person240",
    "cam1_person241.png": "person241",
    "cam1_person242.png": "person242",
    "cam1_person243.png": "person243",
    "cam1_person244.png": "person244",
    "cam1_person245.png": "person245",
    "cam1_person246.png": "person246",
    "cam1_person251.png": "person251",
    "cam1_person252.png": "person252",
    "cam1_person253.png": "person253",
    "cam1_person254.png": "person254",
    "cam1_person255.png": "person255",
    "cam1_person256.png": "person256",
    "cam1_person257.png": "person257",
    "cam1_person258.png": "person258",
    "cam1_person260.png": "person260",
    "cam1_person261.png": "person261",
    "cam1_person262.png": "person262",
    "cam1_person264.png": "person264",
    "cam1_person265.png": "person265",
    "cam1_person266.png": "person266",
    "cam1_person268.png": "person268",
    "cam1_person269.png": "person269",
    "cam1_person270.png": "person270",
    "cam1_person271.png": "person271",
    "cam1_person272.png": "person272",
    "cam1_person273.png": "person273",
    "cam1_person274.png": "person274",
    "cam1_person275.png": "person275",
    "cam1_person276.png": "person276",
    "cam1_person278.png": "person278",
    "cam1_person279.png": "person279",
    "cam1_person280.png": "person280",
    "cam1_person281.png": "person281",
    "cam1_person282.png": "person282",
    "cam1_person284.png": "person284",
    "cam1_person285.png": "person285",
    "cam1_person286.png": "person286",
    "cam1_person287.png": "person287",
    "cam1_person288.png": "person288",
    "cam1_person289.png": "person289",
    "cam1_person290.png": "person290",
    "cam1_person293.png": "person293",
    "cam1_person294.png": "person294",
    "cam1_person295.png": "person295",
    "cam1_person296.png": "person296",
    "cam1_person297.png": "person297",
    "cam1_person298.png": "person298",
    "cam1_person299.png": "person299",
    "cam1_person300.png": "person300",
    "cam1_person301.png": "person301",
    "cam1_person302.png": "person302",
    "cam1_person303.png": "person303",
    "cam1_person304.png": "person304",
    "cam1_person307.png": "person307",
    "cam1_person308.png": "person308",
    "cam1_person309.png": "person309",
    "cam1_person310.png": "person310",
    "cam1_person311.png": "person311",
    "cam1_person312.png": "person312",
    "cam1_person313.png": "person313",
    "cam1_person314.png": "person314",
    "cam1_person315.png": "person315",
    "cam1_person316.png": "person316",
    "cam1_person317.png": "person317",
    "cam1_person319.png": "person319"

}

def get_top_matches(query_features, features_dict_2, top_n=10):
    distance_scores = []
    for person_id, features in features_dict_2.items():
        stored_features = np.mean(features, axis=0)  # Average features for consistency
        distance = cosine(query_features.flatten(), stored_features.flatten())  # Cosine distance
        distance_scores.append((person_id, distance))

    distance_scores.sort(key=lambda x: x[1])  # Sort by lowest distance
    return [match[0] for match in distance_scores[:top_n]]  # Return top N matching person IDs

# Compute Rank-1, Rank-5, and Rank-10 accuracy
rank1_correct = 0
rank5_correct = 0
rank10_correct = 0
total_queries = len(query_labels)

for query_image_name, true_person_id in query_labels.items():
    query_image_path = os.path.join(query_folder_path, query_image_name)
    query_features = extract_features(query_image_path, resnet50, transform, device)
    top_matches = get_top_matches(query_features, features_dict_2, top_n=10)

    if true_person_id == top_matches[0]:
        rank1_correct += 1
    if true_person_id in top_matches[:5]:
        rank5_correct += 1
    if true_person_id in top_matches[:10]:
        rank10_correct += 1

rank1_accuracy = rank1_correct / total_queries
rank5_accuracy = rank5_correct / total_queries
rank10_accuracy = rank10_correct / total_queries

print(f"Rank-1 Accuracy: {rank1_accuracy:.2%}")
print(f"Rank-5 Accuracy: {rank5_accuracy:.2%}")
print(f"Rank-10 Accuracy: {rank10_accuracy:.2%}")



In [ ]:
import numpy as np
import os
from scipy.spatial.distance import cosine

def compute_average_precision(true_id, retrieved_ids):
    """Computes the Average Precision (AP) for a single query."""
    relevant = [1 if str(pid) == str(true_id) else 0 for pid in retrieved_ids]

    num_relevant = sum(relevant)
    if num_relevant == 0:
        return 0.0

    cum_sum = 0.0
    relevant_count = 0
    for i, rel in enumerate(relevant):
        if rel == 1:
            relevant_count += 1
            cum_sum += relevant_count / (i + 1)

    return cum_sum / num_relevant

def compute_map(query_folder_path, query_labels, features_dict, model, transform, device, top_n=10):
    """Computes Mean Average Precision (mAP) over all queries."""
    average_precisions = []

    for query_image_name, true_person_id in query_labels.items():
        query_image_path = os.path.join(query_folder_path, query_image_name)
        query_features = extract_features(query_image_path, model, transform, device)

        top_matches = get_top_matches(query_features, features_dict, top_n=top_n)

        ap = compute_average_precision(true_person_id, top_matches)
        average_precisions.append(ap)

    mean_ap = np.mean(average_precisions) if average_precisions else 0.0
    print(f"Mean Average Precision (mAP): {mean_ap:.2%}")
    return mean_ap

query_labels={
    "cam1_person001.png": "person001",
    "cam1_person002.png": "person002",
    "cam1_person003.png": "person003",
    "cam1_person005.png": "person005",
    "cam1_person006.png": "person006",
    "cam1_person007.png": "person007",
    "cam1_person008.png": "person008",
    "cam1_person010.png": "person010",
    "cam1_person012.png": "person012",
    "cam1_person013.png": "person013",
    "cam1_person014.png": "person014",
    "cam1_person015.png": "person015",
    "cam1_person016.png": "person016",
    "cam1_person017.png": "person017",
    "cam1_person018.png": "person018",
    "cam1_person019.png": "person019",
    "cam1_person020.png": "person020",
    "cam1_person022.png": "person022",
    "cam1_person023.png": "person023",
    "cam1_person024.png": "person024",
    "cam1_person025.png": "person025",
    "cam1_person026.png": "person026",
    "cam1_person027.png": "person027",
    "cam1_person028.png": "person028",
    "cam1_person029.png": "person029",
    "cam1_person031.png": "person031",
    "cam1_person034.png": "person034",
    "cam1_person035.png": "person035",
    "cam1_person036.png": "person036",
    "cam1_person037.png": "person037",
    "cam1_person038.png": "person038",
    "cam1_person039.png": "person039",
    "cam1_person040.png": "person040",
    "cam1_person041.png": "person041",
    "cam1_person042.png": "person042",
    "cam1_person046.png": "person046",
    "cam1_person048.png": "person048",
    "cam1_person049.png": "person049",
    "cam1_person050.png": "person050",
    "cam1_person051.png": "person051",
    "cam1_person053.png": "person053",
    "cam1_person054.png": "person054",
    "cam1_person055.png": "person055",
    "cam1_person057.png": "person057",
    "cam1_person058.png": "person058",
    "cam1_person059.png": "person059",
    "cam1_person060.png": "person060",
    "cam1_person062.png": "person062",
    "cam1_person064.png": "person064",
    "cam1_person066.png": "person066",
    "cam1_person067.png": "person067",
    "cam1_person068.png": "person068",
    "cam1_person069.png": "person069",
    "cam1_person070.png": "person070",
    "cam1_person071.png": "person071",
    "cam1_person075.png": "person075",
    "cam1_person076.png": "person076",
    "cam1_person079.png": "person079",
    "cam1_person080.png": "person080",
    "cam1_person081.png": "person081",
    "cam1_person083.png": "person083",
    "cam1_person084.png": "person084",
    "cam1_person085.png": "person085",
    "cam1_person086.png": "person086",
    "cam1_person087.png": "person087",
    "cam1_person088.png": "person088",
    "cam1_person091.png": "person091",
    "cam1_person092.png": "person092",
    "cam1_person093.png": "person093",
    "cam1_person094.png": "person094",
    "cam1_person095.png": "person095",
    "cam1_person096.png": "person096",
    "cam1_person097.png": "person097",
    "cam1_person098.png": "person098",
    "cam1_person099.png": "person099",
    "cam1_person100.png": "person100",
    "cam1_person102.png": "person102",
    "cam1_person103.png": "person103",
    "cam1_person104.png": "person104",
    "cam1_person105.png": "person105",
    "cam1_person107.png": "person107",
    "cam1_person108.png": "person108",
    "cam1_person109.png": "person109",
    "cam1_person113.png": "person113",
    "cam1_person114.png": "person114",
    "cam1_person117.png": "person117",
    "cam1_person118.png": "person118",
    "cam1_person119.png": "person119",
    "cam1_person120.png": "person120",
    "cam1_person121.png": "person121",
    "cam1_person123.png": "person123",
    "cam1_person124.png": "person124",
    "cam1_person128.png": "person128",
    "cam1_person129.png": "person129",
    "cam1_person130.png": "person130",
    "cam1_person131.png": "person131",
    "cam1_person133.png": "person133",
    "cam1_person134.png": "person134",
    "cam1_person135.png": "person135",
    "cam1_person136.png": "person136",
    "cam1_person137.png": "person137",
    "cam1_person142.png": "person142",
    "cam1_person143.png": "person143",
    "cam1_person145.png": "person145",
    "cam1_person147.png": "person147",
    "cam1_person150.png": "person150",
    "cam1_person151.png": "person151",
    "cam1_person152.png": "person152",
    "cam1_person153.png": "person153",
    "cam1_person154.png": "person154",
    "cam1_person155.png": "person155",
    "cam1_person157.png": "person157",
    "cam1_person159.png": "person159",
    "cam1_person160.png": "person160",
    "cam1_person162.png": "person162",
    "cam1_person163.png": "person163",
    "cam1_person164.png": "person164",
    "cam1_person165.png": "person165",
    "cam1_person166.png": "person166",
    "cam1_person167.png": "person167",
    "cam1_person168.png": "person168",
    "cam1_person169.png": "person169",
    "cam1_person170.png": "person170",
    "cam1_person171.png": "person171",
    "cam1_person172.png": "person172",
    "cam1_person173.png": "person173",
    "cam1_person174.png": "person174",
    "cam1_person175.png": "person175",
    "cam1_person176.png": "person176",
    "cam1_person177.png": "person177",
    "cam1_person179.png": "person179",
    "cam1_person181.png": "person181",
    "cam1_person182.png": "person182",
    "cam1_person183.png": "person183",
    "cam1_person185.png": "person185",
    "cam1_person187.png": "person187",
    "cam1_person189.png": "person189",
    "cam1_person191.png": "person191",
    "cam1_person192.png": "person192",
    "cam1_person193.png": "person193",
    "cam1_person195.png": "person195",
    "cam1_person196.png": "person196",
    "cam1_person197.png": "person197",
    "cam1_person198.png": "person198",
    "cam1_person200.png": "person200",
    "cam1_person202.png": "person202",
    "cam1_person203.png": "person203",
    "cam1_person204.png": "person204",
    "cam1_person205.png": "person205",
    "cam1_person206.png": "person206",
    "cam1_person207.png": "person207",
    "cam1_person208.png": "person208",
    "cam1_person209.png": "person209",
    "cam1_person210.png": "person210",
    "cam1_person212.png": "person212",
    "cam1_person213.png": "person213",
    "cam1_person214.png": "person214",
    "cam1_person216.png": "person216",
    "cam1_person217.png": "person217",
    "cam1_person218.png": "person218",
    "cam1_person219.png": "person219",
    "cam1_person220.png": "person220",
    "cam1_person221.png": "person221",
    "cam1_person222.png": "person222",
    "cam1_person223.png": "person223",
    "cam1_person224.png": "person224",
    "cam1_person225.png": "person225",
    "cam1_person226.png": "person226",
    "cam1_person227.png": "person227",
    "cam1_person228.png": "person228",
    "cam1_person229.png": "person229",
    "cam1_person230.png": "person230",
    "cam1_person231.png": "person231",
    "cam1_person232.png": "person232",
    "cam1_person233.png": "person233",
    "cam1_person234.png": "person234",
    "cam1_person235.png": "person235",
    "cam1_person237.png": "person237",
    "cam1_person238.png": "person238",
    "cam1_person239.png": "person239",
    "cam1_person240.png": "person240",
    "cam1_person241.png": "person241",
    "cam1_person242.png": "person242",
    "cam1_person243.png": "person243",
    "cam1_person244.png": "person244",
    "cam1_person245.png": "person245",
    "cam1_person246.png": "person246",
    "cam1_person251.png": "person251",
    "cam1_person252.png": "person252",
    "cam1_person253.png": "person253",
    "cam1_person254.png": "person254",
    "cam1_person255.png": "person255",
    "cam1_person256.png": "person256",
    "cam1_person257.png": "person257",
    "cam1_person258.png": "person258",
    "cam1_person260.png": "person260",
    "cam1_person261.png": "person261",
    "cam1_person262.png": "person262",
    "cam1_person264.png": "person264",
    "cam1_person265.png": "person265",
    "cam1_person266.png": "person266",
    "cam1_person268.png": "person268",
    "cam1_person269.png": "person269",
    "cam1_person270.png": "person270",
    "cam1_person271.png": "person271",
    "cam1_person272.png": "person272",
    "cam1_person273.png": "person273",
    "cam1_person274.png": "person274",
    "cam1_person275.png": "person275",
    "cam1_person276.png": "person276",
    "cam1_person278.png": "person278",
    "cam1_person279.png": "person279",
    "cam1_person280.png": "person280",
    "cam1_person281.png": "person281",
    "cam1_person282.png": "person282",
    "cam1_person284.png": "person284",
    "cam1_person285.png": "person285",
    "cam1_person286.png": "person286",
    "cam1_person287.png": "person287",
    "cam1_person288.png": "person288",
    "cam1_person289.png": "person289",
    "cam1_person290.png": "person290",
    "cam1_person293.png": "person293",
    "cam1_person294.png": "person294",
    "cam1_person295.png": "person295",
    "cam1_person296.png": "person296",
    "cam1_person297.png": "person297",
    "cam1_person298.png": "person298",
    "cam1_person299.png": "person299",
    "cam1_person300.png": "person300",
    "cam1_person301.png": "person301",
    "cam1_person302.png": "person302",
    "cam1_person303.png": "person303",
    "cam1_person304.png": "person304",
    "cam1_person307.png": "person307",
    "cam1_person308.png": "person308",
    "cam1_person309.png": "person309",
    "cam1_person310.png": "person310",
    "cam1_person311.png": "person311",
    "cam1_person312.png": "person312",
    "cam1_person313.png": "person313",
    "cam1_person314.png": "person314",
    "cam1_person315.png": "person315",
    "cam1_person316.png": "person316",
    "cam1_person317.png": "person317",
    "cam1_person319.png": "person319"

}
mean_ap = compute_map("/content/Dataset/query", query_labels, features_dict_2, resnet50, transform, device)


In [ ]:
# Accuracy values
rank1_acc = 75.61
rank5_acc = 90.24
rank10_acc = 93.90
# Ranks
ranks = [1, 5, 10]
accuracies = [rank1_acc, rank5_acc, rank10_acc]

# Plot accuracy trends
plt.figure(figsize=(8, 5))
plt.plot(ranks, accuracies, marker='o', linestyle='-', color='b', label='Accuracy')
plt.xlabel('Rank')
plt.ylabel('Accuracy (%)')
plt.title('Rank-wise Accuracy Curve')
plt.xticks(ranks)
plt.yticks(range(50, 101, 5))
plt.ylim(50, 105)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()


In [ ]:
#create the dataframe and show rank1,rank5,rank10 and map score of swin transformer,resnet50 and vgg19 models

import pandas as pd

data = {
    'Model': ['Swin Transformer', 'ResNet50'],
    'Rank 1': [95.12, 75.61],
    'Rank 5': [97.97, 90.24],
    'Rank 10': [99.19, 93.90],
    'mAP Score': [96.27, 81.83]
}

df = pd.DataFrame(data)
df

In [ ]:


import matplotlib.pyplot as plt
import pandas as pd
data = {
    'Model': ['Swin Transformer', 'ResNet50'],
    'Rank 1': [95.12, 75.61],
    'Rank 5': [97.97, 90.24],
    'Rank 10': [99.19, 93.90],
    'mAP Score': [96.27, 81.83]
}


df = pd.DataFrame(data)

# Plotting the data
models = df['Model']
rank1 = df['Rank 1']
rank5 = df['Rank 5']
rank10 = df['Rank 10']
map_scores = df['mAP Score']

x = range(len(models))

plt.figure(figsize=(10, 6))

plt.plot(x, rank1, marker='o', label='Rank 1')
plt.plot(x, rank5, marker='s', label='Rank 5')
plt.plot(x, rank10, marker='^', label='Rank 10')
plt.plot(x, map_scores, marker='x', label='mAP Score')

plt.xticks(x, models, rotation=45, ha='right')  # Rotate model names for better readability
plt.xlabel('Model')
plt.ylabel('Accuracy (%) / mAP Score')
plt.title('Rank-wise Accuracy and mAP Score Comparison')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()



